# Scale population and employment for Postcode Sectors and Workplace Zones

In outline:
- get area definitions for Local Authority Districts (LAD - 2011 and 2016), Workplace Zones (WZ), Postcodes (PCD) in Great Britain
- read 2011 census data for workplace population at LAD, WZ
- read 2011 census data for usual resident population at LAD, PCD
- convert 2011 LAD codes to 2016 equivalents
- read population and employment scenarios and baseline projection at LAD scale
- scale future employment to workplace zones
- scale future population to postcode sectors

## Citations

England, Northern Ireland, Scotland and Wales 2011 Census
- Office for National Statistics ; National Records of Scotland ; Northern Ireland Statistics and Research Agency (2017): 2011 Census aggregate data. UK Data Service (Edition: February 2017). DOI: http://dx.doi.org/10.5257/census/aggregate-2011-2
This information is licensed under the terms of the Open Government Licence [http://www.nationalarchives.gov.uk/doc/open-government-licence/version/2].

England and Wales 2001 Census
- Office for National Statistics (2011): 2001 Census aggregate data (Edition: May 2011). UK Data Service. DOI: http://dx.doi.org/10.5257/census/aggregate-2001-2


In [ ]:
import os
import zipfile
from glob import glob

import geopandas as gpd
import pandas as pd
import requests
from geopandas.tools import explicit_crs_from_epsg
from shapely.geometry import Point

## Get boundaries and lookups

In [ ]:
def download(url, filename, dirname=".", force=False):
    if force or not os.path.exists(filename):
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)
    if filename.endswith(".zip"):
        with zipfile.ZipFile(filename,"r") as zf:
            zf.extractall(dirname)    

In [ ]:
# Scotland Workplace Zones
# ref: https://www.nrscotland.gov.uk/statistics-and-data/geography/our-products/census-datasets/2011-census/2011-boundaries
url = "https://www.nrscotland.gov.uk/files//geography/products/workplacezones2011scotland.zip"
download(url, "data/workplacezones2011scotland.zip")
wz_sc = gpd.read_file("data/WorkplaceZones2011Scotland/WorkplaceZones2011Scotland.shp").rename(columns={
    'LADCD': 'lad11cd',
    'WZCD': 'wz11cd'
})
wz_sc.head()

In [ ]:
# England and Wales Workplace Zones
# ref: https://geoportal.statistics.gov.uk/datasets/workplace-zones-december-2011-full-clipped-boundaries-in-england-and-wales
url = "https://opendata.arcgis.com/datasets/a399c2a5922a4beaa080de63c0a218a3_0.zip?outSR=%7B%22latestWkid%22%3A27700%2C%22wkid%22%3A27700%7D"
download(url, "data/workplacezones2011ew.zip", "data")
wz_ew = gpd.read_file("data/Workplace_Zones_December_2011_Full_Clipped_Boundaries_in_England_and_Wales.shp")[
    ['lad11cd', 'wz11cd', 'geometry']
]
wz_ew.head()

## Get 2011 census data 
- population by Postcode Sector, LAD
- workplace population by Workplace Zone, LAD

In [ ]:
def unique_geo_len_sum(df):
    return len(df.GEOGRAPHY_CODE.unique()) == len(df), len(df), df.OBS_VALUE.sum()

### Postcode resident population
Postcode data sourced from Nomis and Scotland Census

- ref https://www.nomisweb.co.uk/census/2011/postcode_headcounts_and_household_estimates
- data https://www.nomisweb.co.uk/output/census/2011/Postcode_Estimates_Table_1.csv


- ref https://www.scotlandscensus.gov.uk/bulletin-figures-and-tables
- data https://www.scotlandscensus.gov.uk/documents/censusresults/release1c/rel1c2tableA1.csv

Cleaned by Marcus Young, Southampton University

In [ ]:
pcd_pop11 = pd.read_csv("data/pcpop11.csv.gz").sort_values('postcode').rename(columns={
    'oslaua': 'lad11cd',
    'population': 'pop11'
})
pcd_pop11.head()

In [ ]:
pcd_pop11.pop11.sum()

In [ ]:
lad_pop11 = pcd_pop11.groupby('lad11cd').sum().reset_index().rename(columns={
    'pop11': 'ladpop11'
})

In [ ]:
pcd_pop11 = pcd_pop11.merge(lad_pop11, on='lad11cd', how='left')

In [ ]:
pcd_pop11['proportion11'] = pcd_pop11.pop11 / pcd_pop11.ladpop11

### Workplace zone workplace population
Workplace population density is available by workplace zones through NOMIS
https://www.nomisweb.co.uk/census/2011/wp102ew

In [ ]:
# Scotland
url = 'http://www.scotlandscensus.gov.uk/documents/additional_tables/WP103SCwz.csv'
download(url, "data/WP103SCwz.csv", "data")

# England and Wales - ref https://www.nomisweb.co.uk/census/2011/wp102ew
# Download via javascript

In [ ]:
unique_geo_len_sum(wp_wz11_ew)

## Scale and project scenarios